In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
# loading excel sheets, combining all sheets for two excel files
forest_file = "Bird_Monitoring_Data_FOREST.XLSX"
grassland_file = "Bird_Monitoring_Data_GRASSLAND.XLSX"
def merge_non_empty_sheets(file_path):
    xls = pd.ExcelFile(file_path)
    dfs = []

    for sheet in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet)
        if not df.empty:  # Only add non-empty sheets
            df["Source_Sheet"] = sheet  # Add sheet name for reference
            dfs.append(df)

    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

# Load and merge non-empty sheets
forest_data = merge_non_empty_sheets(forest_file)
grassland_data = merge_non_empty_sheets(grassland_file)

# Display dataset heads
print("Forest Data:")
print(forest_data.head(5), "\n")
print("Grassland Data:")
print(grassland_data.head(5), "\n")

Forest Data:
  Admin_Unit_Code Sub_Unit_Code Site_Name  Plot_Name Location_Type  Year  \
0            ANTI           NaN    ANTI 1  ANTI-0036        Forest  2018   
1            ANTI           NaN    ANTI 1  ANTI-0036        Forest  2018   
2            ANTI           NaN    ANTI 1  ANTI-0036        Forest  2018   
3            ANTI           NaN    ANTI 1  ANTI-0036        Forest  2018   
4            ANTI           NaN    ANTI 1  ANTI-0036        Forest  2018   

        Date Start_Time  End_Time          Observer  ...  AOU_Code  \
0 2018-05-22   06:19:00  06:29:00  Elizabeth Oswald  ...      EATO   
1 2018-05-22   06:19:00  06:29:00  Elizabeth Oswald  ...      WBNU   
2 2018-05-22   06:19:00  06:29:00  Elizabeth Oswald  ...      RBWO   
3 2018-05-22   06:19:00  06:29:00  Elizabeth Oswald  ...      OROR   
4 2018-05-22   06:19:00  06:29:00  Elizabeth Oswald  ...      NOMO   

  PIF_Watchlist_Status Regional_Stewardship_Status Temperature   Humidity  \
0                False          

In [4]:
#data precrocessing and data cleaning
#finding null values
print("Missing values in Forest Data:\n", forest_data.isnull().sum(), "\n")
print("Missing values in Grassland Data:\n", grassland_data.isnull().sum(), "\n")

Missing values in Forest Data:
 Admin_Unit_Code                   0
Sub_Unit_Code                  7824
Site_Name                         0
Plot_Name                         0
Location_Type                     0
Year                              0
Date                              0
Start_Time                        0
End_Time                          0
Observer                          0
Visit                             0
Interval_Length                   0
ID_Method                         1
Distance                         92
Flyover_Observed                  0
Sex                            5183
Common_Name                       0
Scientific_Name                   0
AcceptedTSN                       9
NPSTaxonCode                      0
AOU_Code                          0
PIF_Watchlist_Status              0
Regional_Stewardship_Status       0
Temperature                       0
Humidity                          0
Sky                               0
Wind                            

In [5]:
# Dropping  empty column Sub_Unit_Code if it exists in both datasets
drop_cols_forest = ["Sub_Unit_Code"] if "Sub_Unit_Code" in forest_data.columns else []
drop_cols_grass = ["Sub_Unit_Code"] if "Sub_Unit_Code" in forest_data.columns else []
forest_data.drop(columns=drop_cols_forest, inplace=True, errors="ignore")
grassland_data.drop(columns=drop_cols_grass, inplace=True, errors="ignore")
forest_data.drop(columns=["Site Name"], inplace=True, errors="ignore")


In [6]:
print(forest_data.isnull().sum(), "\n")
print(grassland_data.isnull().sum(), "\n")

Admin_Unit_Code                   0
Site_Name                         0
Plot_Name                         0
Location_Type                     0
Year                              0
Date                              0
Start_Time                        0
End_Time                          0
Observer                          0
Visit                             0
Interval_Length                   0
ID_Method                         1
Distance                         92
Flyover_Observed                  0
Sex                            5183
Common_Name                       0
Scientific_Name                   0
AcceptedTSN                       9
NPSTaxonCode                      0
AOU_Code                          0
PIF_Watchlist_Status              0
Regional_Stewardship_Status       0
Temperature                       0
Humidity                          0
Sky                               0
Wind                              0
Disturbance                       0
Initial_Three_Min_Cnt       

In [7]:
# making column names of both excels same
forest_data.rename(columns={"NPSTaxonCode": "TaxonCode"}, inplace=True)
grassland_data.rename(columns={"Previously_Obs": "Previously_Observed"}, inplace=True)


In [8]:
# Convert numeric columns
forest_data["Year"] = pd.to_numeric(forest_data["Year"], errors="coerce")
grassland_data["Year"] = pd.to_numeric(grassland_data["Year"], errors="coerce")

forest_data["Visit"] = pd.to_numeric(forest_data["Visit"], errors="coerce")
grassland_data["Visit"] = pd.to_numeric(grassland_data["Visit"], errors="coerce")


In [13]:
# Function to extract numeric value from string
def extract_distance(value):
    if pd.isna(value):
        return np.nan  # Keep NaNs for now
    value = str(value).replace(' Meters', '').strip()  # Ensure it's a string and remove ' Meters'
    
    if '<=' in value:  # Handling values like '<=50'
        return float(value.replace('<=', '').strip())
    elif '-' in value:  # Handling ranges like '50 - 100'
        try:
            start, end = map(float, value.split('-'))  # Convert both to float
            return (start + end) / 2  # Return the average
        except ValueError:
            print(f"Skipping invalid value: {value}")  # Debugging
            return np.nan
    else:
        try:
            return float(value)  # If it's already a number
        except ValueError:
            print(f"Skipping invalid value: {value}")  # Debugging
            return np.nan

# Apply function to Distance column
forest_data['Distance'] = forest_data['Distance'].apply(extract_distance).astype(float)
grassland_data['Distance'] = grassland_data['Distance'].apply(extract_distance).astype(float)

# Check if there are any NaN values before filling them
if forest_data['Distance'].isna().sum() > 0:
    print("NaN values detected in forest_data['Distance'], filling with mean...")
    forest_data['Distance'].fillna(forest_data['Distance'].mean())

if grassland_data['Distance'].isna().sum() > 0:
    print("NaN values detected in grassland_data['Distance'], filling with mean...")
    grassland_data['Distance'].fillna(grassland_data['Distance'].mean())

# Print results for debugging
print(forest_data.head(5))
print(grassland_data.head(5))



  Admin_Unit_Code Site_Name  Plot_Name Location_Type  Year       Date  \
0            ANTI    ANTI 1  ANTI-0036        Forest  2018 2018-05-22   
1            ANTI    ANTI 1  ANTI-0036        Forest  2018 2018-05-22   
2            ANTI    ANTI 1  ANTI-0036        Forest  2018 2018-05-22   
3            ANTI    ANTI 1  ANTI-0036        Forest  2018 2018-05-22   
4            ANTI    ANTI 1  ANTI-0036        Forest  2018 2018-05-22   

  Start_Time  End_Time          Observer  Visit  ... AOU_Code  \
0   06:19:00  06:29:00  Elizabeth Oswald      1  ...     EATO   
1   06:19:00  06:29:00  Elizabeth Oswald      1  ...     WBNU   
2   06:19:00  06:29:00  Elizabeth Oswald      1  ...     RBWO   
3   06:19:00  06:29:00  Elizabeth Oswald      1  ...     OROR   
4   06:19:00  06:29:00  Elizabeth Oswald      1  ...     NOMO   

  PIF_Watchlist_Status  Regional_Stewardship_Status  Temperature   Humidity  \
0                False                         True         19.9  79.400002   
1           

In [10]:
fill_dict = {"Sex": "Unknown", "AcceptedTSN": "Unknown", "TaxonCode": "Unknown", "ID_Method": "Unknown"}
forest_data.fillna(fill_dict,inplace=True)
grassland_data.fillna(fill_dict,inplace=True)

In [11]:
# Remove duplicates
forest_data.drop_duplicates(inplace=True)
grassland_data.drop_duplicates(inplace=True)

In [12]:
# Recheck missing values
print("Missing values in Forest Data:\n", forest_data.isnull().sum(), "\n")
print("Missing values in Grassland Data:\n", grassland_data.isnull().sum(), "\n")

Missing values in Forest Data:
 Admin_Unit_Code                0
Site_Name                      0
Plot_Name                      0
Location_Type                  0
Year                           0
Date                           0
Start_Time                     0
End_Time                       0
Observer                       0
Visit                          0
Interval_Length                0
ID_Method                      0
Distance                       0
Flyover_Observed               0
Sex                            0
Common_Name                    0
Scientific_Name                0
AcceptedTSN                    0
TaxonCode                      0
AOU_Code                       0
PIF_Watchlist_Status           0
Regional_Stewardship_Status    0
Temperature                    0
Humidity                       0
Sky                            0
Wind                           0
Disturbance                    0
Initial_Three_Min_Cnt          0
Source_Sheet                   0
dtype: int6